In [2]:
from tqdm.auto import tqdm
import pandas as pd
from transformers import pipeline
sent_pipeline = pipeline('sentiment-analysis') #Importing the model

C:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [3]:
df = pd.read_csv("Reviews.csv")   #Read csv file
df.head()
df = df[:500]    #Cut the data to only first 500 rows
print(df.shape)

(500, 10)


In [4]:
sent_pipeline("The food was ok")  #Model testing

[{'label': 'POSITIVE', 'score': 0.9998024106025696}]

In [5]:
result = {}
for i , j in tqdm(df.iterrows() , total = len(df)):
    try:
        text = j['Text']
        myid = j['Id']
        result[myid] = sent_pipeline(text)
    except RuntimeError:
        print(f'Broke for id {myid}')
print(result)

  0%|          | 0/500 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (582 > 512). Running this sequence through the model will result in indexing errors


Broke for id 83
Broke for id 187
{1: [{'label': 'POSITIVE', 'score': 0.9983851909637451}], 2: [{'label': 'NEGATIVE', 'score': 0.9995254278182983}], 3: [{'label': 'POSITIVE', 'score': 0.9997652173042297}], 4: [{'label': 'POSITIVE', 'score': 0.9991534948348999}], 5: [{'label': 'POSITIVE', 'score': 0.9987082481384277}], 6: [{'label': 'POSITIVE', 'score': 0.9992551207542419}], 7: [{'label': 'POSITIVE', 'score': 0.9993639588356018}], 8: [{'label': 'POSITIVE', 'score': 0.9998829364776611}], 9: [{'label': 'POSITIVE', 'score': 0.9985174536705017}], 10: [{'label': 'POSITIVE', 'score': 0.9994561076164246}], 11: [{'label': 'POSITIVE', 'score': 0.9361926317214966}], 12: [{'label': 'NEGATIVE', 'score': 0.9992430210113525}], 13: [{'label': 'NEGATIVE', 'score': 0.9899449944496155}], 14: [{'label': 'POSITIVE', 'score': 0.9998890161514282}], 15: [{'label': 'POSITIVE', 'score': 0.9976365566253662}], 16: [{'label': 'POSITIVE', 'score': 0.9992697834968567}], 17: [{'label': 'POSITIVE', 'score': 0.968860566

In [33]:
result_df = pd.DataFrame.from_dict(result,orient = 'index', columns=['result'])
result_df[['label', 'score']] = pd.DataFrame(result_df['result'].tolist(), index=result_df.index)

# Add a column to indicate sentiment
result_df['sentiment'] = result_df['label'].apply(lambda x: 'Positive' if x == 'POSITIVE' else 'Negative')
result_df.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)
# Merge the result DataFrame with the original dataset
merged_dataset = df.merge(result_df ,how = 'left', left_index = True , right_index = True)
# Show the merged dataset
merged_dataset.drop(["result"] , axis = 1)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,label,score,sentiment
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,POSITIVE,0.998385,Positive
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,NEGATIVE,0.999525,Negative
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,POSITIVE,0.999765,Positive
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,POSITIVE,0.999153,Positive
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,POSITIVE,0.998708,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,496,B000G6RYNE,APGAA43E3WPN7,Darren,0,0,5,1201392000,amazing chips,i rarely eat chips but i saw these and tried t...,POSITIVE,0.999425,Positive
496,497,B000G6RYNE,ABR7HU5H1KNE,Keith,0,0,5,1196726400,Best Chip Ever,This is easily the best potato chip that I hav...,POSITIVE,0.992952,Positive
497,498,B000G6RYNE,AJQD2WWJYOYFQ,bubbles,0,0,4,1186617600,"Tangy, spicy, and sweet- oh my!",Kettle Chips Spicy Thai potato chips have the ...,POSITIVE,0.999796,Positive
498,499,B000G6RYNE,A16YH487W9ZYO0,Bruce G. Lindsay,0,0,4,1184198400,An indulgence with a bite,"Okay, I should not eat potato chips, nor shoul...",NaN,NaN,NaN
